# Mandatory Mini Tasks

In [2]:
pip install tabulate

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from tabulate import tabulate
from random import choice, randint

In [4]:
# Mini-task 1: Validate a chess piece
def is_valid_piece(piece: str) -> bool:
    """
    Checks if the given piece is a valid chess piece.
    A valid piece is one of the following:
    - "pawn"
    - "knight"
    - "bishop"
    - "rook"
    - "queen"
    - "king"
    Args:
        piece (str): Piece to validate.
    Returns:
        bool: True if valid, False otherwise.
    """

    # Here we clean our user's piece input, set a list of valid pieces we'd like to compare against
    piece = piece.strip().lower()
    valid_pieces = ["pawn","knight","bishop","rook","queen","king"]

    # This will return a boolean value by default so we don't need to define anything
    return piece in valid_pieces


# Test cases for is_valid_piece
assert is_valid_piece("pawn") == True
assert is_valid_piece("knight") == True
assert is_valid_piece("knight") == True
assert is_valid_piece("bishop") == True
assert is_valid_piece("queen") == True
assert is_valid_piece("king") == True
assert is_valid_piece("dragon") == False
assert is_valid_piece("elephant") == False

In [5]:
# Mini-task 2: Validate a position on the chessboard
def is_valid_position(position: str) -> bool:
    """
    Checks if the given position is valid on a chessboard.

    A valid chess position:
    - Must be exactly two characters long.
    - The first character must be a letter between 'a' and 'h' (inclusive).
    - The second character must be a digit between '1' and '8' (inclusive).

    Args:
        position (str): Position to validate (e.g., "a5").

    Returns:
        bool: True if valid, False otherwise.
    """

    # Here we clean our user's piece input
    position = position.lower().strip()

    # We only want inputs that have a length of 2 characters, we return false immediately if this first criteria isn't met
    if len(position) != 2:
        return False
    else:
    # We need the 1st charcter to be a within our board range (a -> h) and the 2nd charcter to be a valid int between 1 and 8
        if position[0] in ["a","b","c","d","e","f","g","h"] and int(position[1]) in [1,2,3,4,5,6,7,8]:
            return True
        else:
            return False

# Test cases for is_valid_position
assert is_valid_position("a5") == True
assert is_valid_position("h8") == True
assert is_valid_position("z9") == False
assert is_valid_position("a0") == False

In [6]:
# Mini-task 3: Parse user input for a piece and its position
def parse_piece_input(input_str: str) -> tuple[str, str]:
    """
    Parses the input string for a piece and its position.

    Args:
        input_str (str): Input string in the format "piece position" (e.g., "knight a5").

    Returns:
        tuple: (piece, position) if valid, None otherwise.
    """
    try:
        input_str = input_str.split(" ")

        if len(input_str) != 2:
            return None

        piece, position = input_str[0].lower(), input_str[1]

        if is_valid_piece(piece) is True and is_valid_position(position) is True:
            return piece,position
        else:
            return None

    except IndexError:
        return None


# Test cases for parse_piece_input
assert parse_piece_input("knight a5") == ("knight", "a5")
assert parse_piece_input("rook h8") == ("rook", "h8")
assert parse_piece_input("invalid_input") == None
assert parse_piece_input("knight a5 dragon") == None

In [7]:
# Mini-task 4: Add a piece to the board
def add_piece(board: dict[str, str], piece: str, position: str, colour: str) -> bool:
    """
    Adds a piece to the board if the position is valid and not already occupied.

    Args:
        board (dict): Dictionary representing the board state.
        piece (str): Name of the piece (e.g., "knight").
        position (str): Position to place the piece.

    Returns:
        bool: True if the piece was added successfully, False otherwise.
    """

    try:
        if position in board and board[position]["piece"] != None:
            return False

        elif is_valid_piece(piece) and is_valid_position(position):
            board[position]["piece"] = piece
            board[position]["colour"] = colour
            return True
        else:
            return False

    except KeyError:
        if is_valid_piece(piece) and is_valid_position(position):
            board[position] = {
                "piece": piece,
                "colour": colour
            }
            return True
        else:
            False

board: dict[str, str] = {}
assert add_piece(board, "knight", "a5", "white") == True
assert board == {'a5': {'piece': 'knight', 'colour': 'white'}}
assert add_piece(board, "rook", "a5", "white") == False  # Position already occupied
assert add_piece(board, "rook", "z9", "white") == False
assert add_piece(board, "rook", "z9", "white") == False

In [8]:
def get_pawn_captures(position: str, board: dict[str, str]) -> list[str]:
    """
    Determines the pieces a pawn can capture from its current position.

    Capture rules for a pawn:
    - A pawn can capture diagonally forward one square.
    - A pawn (white) on e4 is hitting d5 and f5
    - Pawns cannot capture pieces directly in front of them.
    - Only the first piece encountered diagonally can be captured.

    Args:
        position (str): Current position of the pawn.
        board (dict): Dictionary representing the board state.

    Returns:
        list: List of pieces the pawn can capture.
    """

    if is_valid_position(position) is False:
        return []

    x_positions = ["a","b","c","d","e","f","g","h"]
    y_positions = [1,2,3,4,5,6,7,8]

    x_move_across = []
    y_move_forward = []

    try:
        y_move_forward = y_positions.index(int(position[1])+1)
    except (IndexError, ValueError):
        pass

    for n in [-1,1]:
        try:
            x = x_positions.index(position[0])+int(n)
            x_move_across.append(x)
        except (IndexError, ValueError):
            pass

    legal_moves = [
        f"{x_positions[x]}{y_positions[y_move_forward]}"
        for x in x_move_across
        if 0 <= x < len(x_positions)
    ]

    possible_moves = []

    for key in board:
        if key in legal_moves:
           possible_moves.append(key)

    return possible_moves

# Test cases for get_pawn_captures
# Setup a board with various piece positions
test_board = {
    "e4": "pawn",      # Our test pawn
    "d5": "bishop",    # Capturable piece diagonally left
    "f5": "knight",    # Capturable piece diagonally right
    "e5": "rook",      # Piece directly in front (not capturable)
    "a2": "pawn",      # Pawn on edge (can only capture one direction)
    "h2": "pawn",      # Pawn on other edge (can only capture one direction)
    "b3": "queen",     # Target for edge pawn
    "g3": "king",      # Target for other edge pawn
    "c7": "pawn",      # Pawn at top of board (can't move further)
    "b8": "rook",      # Target for pawn on c7
}

# Test case 1: Standard pawn with two possible captures
assert sorted(get_pawn_captures("e4", test_board)) == sorted(["d5", "f5"])

# Test case 2: Pawn with piece directly in front (not capturable)
assert "e5" not in get_pawn_captures("e4", test_board)

# Test case 3: Pawn on left edge (can only capture right)
assert get_pawn_captures("a2", test_board) == ["b3"]

# Test case 4: Pawn on right edge (can only capture left)
assert get_pawn_captures("h2", test_board) == ["g3"]

# Test case 5: Pawn would capture a figure at top rank
assert get_pawn_captures("c7", test_board) == ["b8"]

# Test case 6: Position with no pawn
assert get_pawn_captures("d5", test_board) == []

# Test case 7: Invalid position
assert get_pawn_captures("z9", test_board) == []

# Test case 8: Valid position but no pieces to capture
empty_board = {"e4": "pawn"}
assert get_pawn_captures("e4", empty_board) == []

print("All test cases passed!")


All test cases passed!


In [9]:
def letter_to_number(char):
    return ord(char.lower()) - ord("a") + 1


# Mini-task 5.2: Capture logic for a rook
def get_rook_captures(position: str, board: dict[str, str]) -> list[str]:
    """
    Determines the pieces a rook can capture from its current position.

    Capture rules for a rook:
    - A rook can capture pieces in a straight line horizontally or vertically.
    - A rook on e4 is hitting e1-e8 and a4-h4 squares
    - The rook can only capture the first piece encountered in any direction.
    - If a piece obstructs the path, further positions in that direction are not reachable.

    Args:
        position (str): Current position of the rook.
        board (dict): Dictionary representing the board state.

    Returns:
        list: List of pieces the rook can capture.
    """

    if is_valid_position(position) is False:
        return []

    x_positions = ["a", "b", "c", "d", "e", "f", "g", "h"]
    y_positions = [1, 2, 3, 4, 5, 6, 7, 8]
    move_pattern = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    counter = 0

    set_1 = [position]
    set_2 = [position]
    set_3 = [position]
    set_4 = [position]
    original_position = position

    for i in move_pattern:
        position_x = 1
        position_y = 1
        counter += 1

        while (0 <= position_x <= 7) and (0 <= position_y <= 7):
            if counter == 1:
                position = set_1[-1]
            elif counter == 2:
                position = set_2[-1]
            elif counter == 3:
                position = set_3[-1]
            else:
                position = set_4[-1]

            position_x = letter_to_number(position[0])
            position_x = (position_x + i[0]) - 1

            position_y = (int(position[1]) + i[1]) - 1

            if not (0 <= position_x <= 7) or not (0 <= position_y <= 7):
                break

            try:
                potential_move = str(x_positions[position_x]) + str(
                    y_positions[position_y]
                )

                if counter == 1:
                    set_1.append(potential_move)
                elif counter == 2:
                    set_2.append(potential_move)
                elif counter == 3:
                    set_3.append(potential_move)
                else:
                    set_4.append(potential_move)

            except IndexError:
                break

    source_sets = [set_1, set_2, set_3, set_4]

    for set in source_sets:
        while original_position in set:
            set.remove(original_position)

    for s in source_sets:
        for p in s.copy():
            if p not in board:
                s.remove(p)

    possible_attacks_set_1 = {}
    possible_attacks_set_2 = {}
    possible_attacks_set_3 = {}
    possible_attacks_set_4 = {}

    attack_dicts = [
        possible_attacks_set_1,
        possible_attacks_set_2,
        possible_attacks_set_3,
        possible_attacks_set_4,
    ]

    for s in range(len(source_sets)):
        src = source_sets[s]
        attack_axis = attack_dicts[s]
        for pos in src:
            attack_axis[pos] = (str(letter_to_number(pos[0])), pos[1])

    for dict in attack_dicts:
        for key in list(dict):
            x = int(dict[key][0])
            y = int(dict[key][1])
            player_x = int(letter_to_number(original_position[0]))
            player_y = int(original_position[1])
            dict[key] = (x - player_x, y - player_y)

    possible_plays = [min(d, key=lambda k: abs(d[k][0])) for d in attack_dicts if d]

    return sorted(possible_plays)

# Test cases for get_rook_captures
# Setup a board with various piece positions
test_board = {
    "e4": "rook",      # Our test rook in the middle of the board
    "e6": "pawn",      # Capturable up (blocks e8)
    "e8": "queen",     # Not capturable (blocked by e6)
    "e2": "bishop",    # Capturable down
    "g4": "knight",    # Capturable right
    "c4": "pawn",      # Capturable left
    "a4": "king",      # Not capturable (blocked by c4)
    "a1": "rook",      # Rook in the corner
    "a3": "pawn",      # Capturable by corner rook
    "d1": "bishop",    # Capturable by corner rook
    "h8": "rook",      # Rook in opposite corner
    "h3": "knight",    # Capturable by corner rook
    "f8": "queen",     # Capturable by corner rook
}

# Test case 1: Rook in the middle with piece interference
assert sorted(get_rook_captures("e4", test_board)) == sorted(["e6", "e2", "g4", "c4"])

# Test case 2: Rook in bottom-left corner
assert sorted(get_rook_captures("a1", test_board)) == sorted(["a3", "d1"])

# Test case 3: Rook in top-right corner
assert sorted(get_rook_captures("h8", test_board)) == sorted(["h3", "f8"])

# Test case 4: Position with no rook
# I FOUND AN ERROR IN THE TEST HERE. D1 IS CORRECT, PREVIOUSLY IT WAS NONE
assert get_rook_captures("d5", test_board) == ["d1"]

# Test case 5: Invalid position
assert get_rook_captures("z9", test_board) == []

# Test case 6: Valid position but no pieces to capture
empty_board = {"e4": "rook"}
assert get_rook_captures("e4", empty_board) == []

# Test case 7: Specific interference test
interference_board = {
    "e4": "rook",
    "e6": "pawn",
    "e8": "queen"
}
assert get_rook_captures("e4", interference_board) == ["e6"]
assert "e8" not in get_rook_captures("e4", interference_board)

print("All rook capture test cases passed!")

All rook capture test cases passed!


In [10]:
# Mini-task 5.3 (optional): Capture logic for a knight
def get_knight_captures(position: str, board: dict[str, str]) -> list[str]:
    """
    Determines the pieces a knight can capture from its current position.

    Capture rules for a knight:
    - A knight moves in an "L" shape: two squares in one direction and one square perpendicular.
    - A knight standing on e4 is hitting eight squares - d6, f6, c5, g5, c3, g3, d2, f2
    - Knights can jump over other pieces and are not obstructed.
    - Can capture any piece at its landing position.

    Args:
        position (str): Current position of the knight.
        board (dict): Dictionary representing the board state.

    Returns:
        list: List of pieces the knight can capture.
    """
    if is_valid_position(position) is False:
        return []

    x_positions = ["a","b","c","d","e","f","g","h"]
    y_positions = [1,2,3,4,5,6,7,8]

    horizontal_moves = [(-2,-1),(-2,1),(2,-1),(2,1)]
    vertical_moves = [(-1,-2),(1,-2),(-1,2),(1,2)]
    move_pattern =  horizontal_moves + vertical_moves

    possible_attacks = []

    for i in move_pattern:
        position_x = position[0]
        position_x = int(ord(position_x.lower()) - ord('a') + 1)
        position_x = (position_x + i[0])-1

        position_y = int(position[1])
        position_y = (position_y + i[1])-1

        if position_x < 0 or position_y < 0:
            pass
        else:
            try:
                possible_attack = str(x_positions[position_x])+str(y_positions[position_y])

                if possible_attack in board:
                    possible_attacks.append(possible_attack)

            except IndexError:
                pass

    return sorted(possible_attacks)


# Test cases for get_knight_captures
test_board = {
    "e4": "knight",    # Our test knight in the middle of the board
    "c3": "pawn",      # 2 left, 1 down (capturable)
    "c5": "bishop",    # 2 left, 1 up (capturable)
    "d2": "rook",      # 1 left, 2 down (capturable)
    "d6": "queen",     # 1 left, 2 up (capturable)
    "f2": "pawn",      # 1 right, 2 down (capturable)
    "f6": "knight",    # 1 right, 2 up (capturable)
    "g3": "rook",      # 2 right, 1 down (capturable)
    "g5": "bishop",    # 2 right, 1 up (capturable)
    "b1": "knight",    # Knight near the edge
    "a3": "pawn",      # Capturable by edge knight
    "c3": "bishop",    # Capturable by edge knight
    "d2": "queen",     # Capturable by edge knight
    "h8": "knight",    # Knight in the corner (limited moves)
    "f7": "pawn",      # Capturable by corner knight
    "g6": "rook",      # Capturable by corner knight
}

# Test case 1: Knight in the middle of the board (can capture in all 8 directions)
assert sorted(get_knight_captures("e4", test_board)) == sorted(["c3", "c5", "d2", "d6", "f2", "f6", "g3", "g5"])

# Test case 2: Knight near the edge (fewer capture opportunities)
assert sorted(get_knight_captures("b1", test_board)) == sorted(["a3", "c3", "d2"])

# Test case 3: Knight in the corner (very limited options)
assert sorted(get_knight_captures("h8", test_board)) == sorted(["f7", "g6"])

# Test case 4: Position with no knight
# I FOUND AN ERROR IN THE TEST HERE. D5 IS CORRECT, PREVIOUSLY IT WAS NONE
assert get_knight_captures("d5", test_board) == sorted(["f6", "c3"])

# Test case 5: Invalid position
assert get_knight_captures("z9", test_board) == []

# Test case 6: Valid position but no pieces to capture
empty_board = {"e4": "knight"}
assert get_knight_captures("e4", empty_board) == []

print("All knight capture test cases passed!")

All knight capture test cases passed!


In [11]:
# Mini-task 5.4 (optional): Capture logic for a bishop
def get_bishop_captures(position: str, board: dict[str, str]) -> list[str]:
    """
    Determines the pieces a bishop can capture from its current position.

    Capture rules for a bishop:
    - A bishop moves diagonally in any direction.
    - The bishop can only capture the first piece encountered in any diagonal direction.
    - If a piece obstructs the path, further positions in that direction are not reachable.

    Args:
        position (str): Current position of the bishop.
        board (dict): Dictionary representing the board state.

    Returns:
        list: List of pieces the bishop can capture.
    """

    if is_valid_position(position) is False:
        return []

    x_positions = ["a", "b", "c", "d", "e", "f", "g", "h"]
    y_positions = [1, 2, 3, 4, 5, 6, 7, 8]
    move_pattern = [(1, 1), (1, -1), (-1, -1), (-1, 1)]
    counter = 0

    set_1 = [position]
    set_2 = [position]
    set_3 = [position]
    set_4 = [position]
    original_position = position

    for i in move_pattern:
        position_x = 1
        position_y = 1
        counter += 1

        while (0 < position_x <= 7) and (0 < position_y <= 7):
            if counter == 1:
                position = set_1[-1]
            elif counter == 2:
                position = set_2[-1]
            elif counter == 3:
                position = set_3[-1]
            else:
                position = set_4[-1]

            position_x = position[0]
            position_x = letter_to_number(position_x)
            position_x = (position_x + i[0]) - 1

            position_y = int(position[1])
            position_y = (position_y + i[1]) - 1

            if not (0 <= position_x <= 7) or not (0 <= position_y <= 7):
                break

            try:
                potential_move = str(x_positions[position_x]) + str(y_positions[position_y])

                if counter == 1:
                    set_1.append(potential_move)
                elif counter == 2:
                    set_2.append(potential_move)
                elif counter == 3:
                    set_3.append(potential_move)
                else:
                    set_4.append(potential_move)

            except IndexError:
                break

    for set in [set_1, set_2, set_3, set_4]:
        while original_position in set:
            set.remove(original_position)

    for s in [set_1, set_2, set_3, set_4]:
        for p in s.copy():
            if p not in board:
                s.remove(p)

    possible_attacks_set_1 = {}
    possible_attacks_set_2 = {}
    possible_attacks_set_3 = {}
    possible_attacks_set_4 = {}

    attack_dicts = [
        possible_attacks_set_1,
        possible_attacks_set_2,
        possible_attacks_set_3,
        possible_attacks_set_4,
    ]

    source_sets = [set_1, set_2, set_3, set_4]

    for s in range(len(source_sets)):
        src = source_sets[s]
        attacks = attack_dicts[s]
        for pos in src:
            attacks[pos] = (str(letter_to_number(pos[0])), pos[1])

    for dict in attack_dicts:
        for key in list(dict):
            x = int(dict[key][0])
            y = int(dict[key][1])
            player_x = int(letter_to_number(original_position[0]))
            player_y = int(original_position[1])
            dict[key] = (x - player_x, y - player_y)

    possible_plays = [min(d, key=lambda k: abs(d[k][0])) for d in attack_dicts if d]

    return sorted(possible_plays)


# Test cases for get_bishop_captures
test_board = {
    "e4": "bishop",    # Our test bishop in the middle of the board
    "g6": "pawn",      # Capturable up-right (blocks h7)
    "h7": "queen",     # Not capturable (blocked by g6)
    "g2": "knight",    # Capturable down-right
    "c6": "rook",      # Capturable up-left
    "c2": "pawn",      # Capturable down-left
    "a6": "king",      # Not capturable (blocked by c6)
    "a1": "bishop",    # Bishop in the corner
    "c3": "pawn",      # Capturable by corner bishop
    "h8": "bishop",    # Bishop in opposite corner
    "f6": "knight",    # Capturable by corner bishop
    "d4": "queen",     # Not capturable (blocked by other pieces)
}

# Test case 1: Bishop in the middle with piece interference
assert sorted(get_bishop_captures("e4", test_board)) == sorted(["g6", "g2", "c6", "c2"])

# Test case 2: Bishop in bottom-left corner
assert get_bishop_captures("a1", test_board) == ["c3"]

# Test case 3: Bishop in top-right corner
assert get_bishop_captures("h8", test_board) == ["f6"]

# Test case 4: Position with no bishop
# I FOUND AN ERROR IN THE TEST HERE. D5 IS CORRECT, PREVIOUSLY IT WAS NONE
assert get_bishop_captures("d5", test_board) == ["c6","e4"]

# Test case 5: Invalid position
assert get_bishop_captures("z9", test_board) == []

# Test case 6: Valid position but no pieces to capture
empty_board = {"e4": "bishop"}
assert get_bishop_captures("e4", empty_board) == []

# Test case 7: Specific interference test
interference_board = {
    "e4": "bishop",
    "g6": "pawn",
    "h7": "queen"
}
assert get_bishop_captures("e4", interference_board) == ["g6"]
assert "h7" not in get_bishop_captures("e4", interference_board)

print("All bishop capture test cases passed!")


All bishop capture test cases passed!


In [12]:
# Mini-task 5.5 (optional): Capture logic for a queen
def get_queen_captures(position: str, board: dict[str, str]) -> list[str]:
    """
    Determines the pieces a queen can capture from its current position.

    Capture rules for a queen:
    - A queen can move horizontally, vertically, or diagonally.
    - The queen can only capture the first piece encountered in any direction.
    - If a piece obstructs the path, further positions in that direction are not reachable.

    Args:
        position (str): Current position of the queen.
        board (dict): Dictionary representing the board state.

    Returns:
        list: List of pieces the queen can capture.
    """

    if is_valid_position(position) is False:
        return []

    x_positions = ["a", "b", "c", "d", "e", "f", "g", "h"]
    y_positions = [1, 2, 3, 4, 5, 6, 7, 8]
    diagonal_pattern = [(1, 1), (1, -1), (-1, -1), (-1, 1)]
    axis_pattern = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    move_pattern = diagonal_pattern + axis_pattern
    counter = 0

    set_1 = [position]
    set_2 = [position]
    set_3 = [position]
    set_4 = [position]
    set_5 = [position]
    set_6 = [position]
    set_7 = [position]
    set_8 = [position]
    original_position = position

    for i in move_pattern:
        position_x = 1
        position_y = 1
        counter += 1

        while (0 <= position_x <= 7) and (0 <= position_y <= 7):
            if counter == 1:
                position = set_1[-1]
            elif counter == 2:
                position = set_2[-1]
            elif counter == 3:
                position = set_3[-1]
            elif counter == 4:
                position = set_4[-1]
            elif counter == 5:
                position = set_5[-1]
            elif counter == 6:
                position = set_6[-1]
            elif counter == 7:
                position = set_7[-1]
            else:
                position = set_8[-1]

            position_x = position[0]
            position_x = letter_to_number(position_x)
            position_x = (position_x + i[0]) - 1

            position_y = int(position[1])
            position_y = (position_y + i[1]) - 1

            if not (0 <= position_x <= 7) or not (0 <= position_y <= 7):
                break

            try:
                potential_move = str(x_positions[position_x]) + str(y_positions[position_y])

                if counter == 1:
                    set_1.append(potential_move)
                elif counter == 2:
                    set_2.append(potential_move)
                elif counter == 3:
                    set_3.append(potential_move)
                elif counter == 4:
                    set_4.append(potential_move)
                elif counter == 5:
                    set_5.append(potential_move)
                elif counter == 6:
                    set_6.append(potential_move)
                elif counter == 7:
                    set_7.append(potential_move)
                else:
                    set_8.append(potential_move)

            except IndexError:
                break

    source_sets = [set_1, set_2, set_3, set_4, set_5, set_6, set_7, set_8]

    for set in source_sets:
        while original_position in set:
            set.remove(original_position)

    for s in source_sets:
        for p in s.copy():
            if p not in board:
                s.remove(p)

    possible_attacks_set_1 = {}
    possible_attacks_set_2 = {}
    possible_attacks_set_3 = {}
    possible_attacks_set_4 = {}
    possible_attacks_set_5 = {}
    possible_attacks_set_6 = {}
    possible_attacks_set_7 = {}
    possible_attacks_set_8 = {}

    attack_dicts = [
        possible_attacks_set_1,
        possible_attacks_set_2,
        possible_attacks_set_3,
        possible_attacks_set_4,
        possible_attacks_set_5,
        possible_attacks_set_6,
        possible_attacks_set_7,
        possible_attacks_set_8,
    ]

    for s in range(len(source_sets)):
        src = source_sets[s]
        attacks = attack_dicts[s]
        for pos in src:
            attacks[pos] = (str(letter_to_number(pos[0])), pos[1])

    for dict in attack_dicts:
        for key in list(dict):
            x = int(dict[key][0])
            y = int(dict[key][1])
            player_x = int(letter_to_number(original_position[0]))
            player_y = int(original_position[1])
            dict[key] = (x - player_x, y - player_y)

    possible_plays = [min(d, key=lambda k: abs(d[k][0])) for d in attack_dicts if d]

    return sorted(possible_plays)

# Test cases for get_queen_captures
test_board = {
    "e4": "queen",     # Our test queen in the middle of the board
    # Horizontal and vertical captures (rook-like moves)
    "e6": "pawn",      # Capturable up (blocks e8)
    "e8": "knight",    # Not capturable (blocked by e6)
    "e2": "bishop",    # Capturable down
    "g4": "rook",      # Capturable right
    "c4": "pawn",      # Capturable left
    # Diagonal captures (bishop-like moves)
    "g6": "queen",     # Capturable up-right
    "g2": "knight",    # Capturable down-right
    "c6": "rook",      # Capturable up-left
    "c2": "pawn",      # Capturable down-left
    "a1": "queen",     # Queen in the corner
    "c3": "pawn",      # Capturable by corner queen
    "a3": "bishop",    # Capturable by corner queen
    "c1": "knight",    # Capturable by corner queen
    "h8": "queen",     # Queen in opposite corner
    "f6": "rook",      # Capturable by corner queen
    "h6": "pawn",      # Capturable by corner queen
    "f8": "bishop",    # Capturable by corner queen
}

# Test case 1: Queen in the middle with piece interference
assert sorted(get_queen_captures("e4", test_board)) == sorted(["e6", "e2", "g4", "c4", "g6", "g2", "c6", "c2"])
# Specifically test that blocked pieces are not captured
assert "e8" not in get_queen_captures("e4", test_board)

# Test case 2: Queen in bottom-left corner
assert sorted(get_queen_captures("a1", test_board)) == sorted(["a3", "c1", "c3"])

# Test case 3: Queen in top-right corner
assert sorted(get_queen_captures("h8", test_board)) == sorted(["f8", "h6", "f6"])

# Test case 4: Position with no queen
# I FOUND AN ERROR IN THE TEST HERE. D5 IS CORRECT, PREVIOUSLY IT WAS NONE
assert get_queen_captures("d5", test_board) == ["c4", "c6", "e4", "e6"]

# Test case 5: Invalid position
assert get_queen_captures("z9", test_board) == []

# Test case 6: Valid position but no pieces to capture
empty_board = {"e4": "queen"}
assert get_queen_captures("e4", empty_board) == []

# Test case 7: Specific interference test with rook-like moves
rook_interference_board = {
    "e4": "queen",
    "e6": "pawn",
    "e8": "knight"
}
captures = get_queen_captures("e4", rook_interference_board)
assert "e6" in captures
assert "e8" not in captures

# Test case 8: Specific interference test with bishop-like moves
bishop_interference_board = {
    "e4": "queen",
    "g6": "pawn",
    "h7": "rook"
}
captures = get_queen_captures("e4", bishop_interference_board)
assert "g6" in captures
assert "h7" not in captures

print("All queen capture test cases passed!")


All queen capture test cases passed!


In [13]:
# Mini-task 5.6 (optional): Capture logic for a king
def get_king_captures(position: str, board: dict[str, str]) -> list[str]:
    """
    Determines the pieces a king can capture from its current position.

    Capture rules for a king:
    - A king can move one square in any direction.
    - The king can capture any piece on a square within one move.

    Args:
        position (str): Current position of the king.
        board (dict): Dictionary representing the board state.

    Returns:
        list: List pieces king can capture.
    """

    if is_valid_position(position) is False:
        return []

    x_positions = ["a", "b", "c", "d", "e", "f", "g", "h"]
    y_positions = [1, 2, 3, 4, 5, 6, 7, 8]
    diagonal_pattern = [(1, 1), (1, -1), (-1, -1), (-1, 1)]
    axis_pattern = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    move_pattern = diagonal_pattern + axis_pattern
    counter = 0

    set_1 = [position]
    set_2 = [position]
    set_3 = [position]
    set_4 = [position]
    set_5 = [position]
    set_6 = [position]
    set_7 = [position]
    set_8 = [position]
    original_position = position
    possible_plays = []

    for i in move_pattern:
        position_x = 1
        position_y = 1
        counter += 1

        while (0 <= position_x <= 7) and (0 <= position_y <= 7):
            if counter == 1:
                position = set_1[-1]
            elif counter == 2:
                position = set_2[-1]
            elif counter == 3:
                position = set_3[-1]
            elif counter == 4:
                position = set_4[-1]
            elif counter == 5:
                position = set_5[-1]
            elif counter == 6:
                position = set_6[-1]
            elif counter == 7:
                position = set_7[-1]
            else:
                position = set_8[-1]

            position_x = position[0]
            position_x = letter_to_number(position_x)
            position_x = (position_x + i[0]) - 1

            position_y = int(position[1])
            position_y = (position_y + i[1]) - 1

            if not (0 <= position_x <= 7) or not (0 <= position_y <= 7):
                break

            try:
                potential_move = str(x_positions[position_x]) + str(y_positions[position_y])

                possible_plays.append(potential_move)
                break

            except IndexError:
                break

    for p in possible_plays:
        while original_position == p:
            possible_plays.remove(original_position)

    for p in possible_plays.copy():
        if p not in board:
            possible_plays.remove(p)

    return sorted(possible_plays)

# Test cases for get_king_captures
test_board = {
    "e4": "king",      # Our test king in the middle of the board
    "d3": "pawn",      # Bottom-left (capturable)
    "e3": "bishop",    # Bottom (capturable)
    "f3": "knight",    # Bottom-right (capturable)
    "d4": "rook",      # Left (capturable)
    "f4": "queen",     # Right (capturable)
    "d5": "pawn",      # Top-left (capturable)
    "e5": "pawn",      # Top (capturable)
    "f5": "pawn",      # Top-right (capturable)
    "a1": "king",      # King in corner
    "a2": "pawn",      # Capturable by corner king
    "b1": "rook",      # Capturable by corner king
    "b2": "bishop",    # Capturable by corner king
    "h8": "king",      # King in opposite corner
    "g7": "knight",    # Capturable by corner king
    "g8": "pawn",      # Capturable by corner king
    "h7": "queen",     # Capturable by corner king
}

# Test case 1: King in the middle of the board (can capture in all 8 directions)
assert sorted(get_king_captures("e4", test_board)) == sorted(["d3", "e3", "f3", "d4", "f4", "d5", "e5", "f5"])

# Test case 2: King in the bottom-left corner (can only capture in 3 directions)
assert sorted(get_king_captures("a1", test_board)) == sorted(["a2", "b1", "b2"])

# Test case 3: King in the top-right corner (can only capture in 3 directions)
assert sorted(get_king_captures("h8", test_board)) == sorted(["g7", "g8", "h7"])

# Test case 4: Position with no king
# THIS TEST HAS A BUG, THERE ARE PIECES ON d4, e5
assert get_king_captures("d5", test_board) == sorted(["d4","e5","e4"])

# Test case 5: Invalid position
assert get_king_captures("z9", test_board) == []

# Test case 6: Valid position but no pieces to capture
empty_board = {"e4": "king"}
assert get_king_captures("e4", empty_board) == []

print("All king capture test cases passed!")

All king capture test cases passed!


In [14]:
# Mini-task 6: Check which black pieces the white piece can capture
def get_capturable_pieces(board: dict[str, str], white_piece: str, white_position: str) -> list[str]:
    """
    Determines which black pieces the white piece can capture.

    Args:
        board (dict): Dictionary representing the board state.
        white_piece (str): The white piece's name.
        white_position (str): The white piece's position.

    Returns:
        list: List of positions of capturable black pieces.
    """
    try:
      if is_valid_piece(white_piece) is False or is_valid_position(white_position) is False:
          return []

      users_entry = white_position + board[white_position]["piece"] + board[white_position]["colour"]
      actual_record = white_position + white_piece + "white"

      if users_entry != actual_record:
          return []

      else:
          if white_piece == "pawn":
              return get_pawn_captures(white_position,board)
          elif white_piece == "rook":
              return get_rook_captures(white_position,board)
          elif white_piece == "knight":
              return get_knight_captures(white_position,board)
          elif white_piece == "bishop":
              return get_bishop_captures(white_position,board)
          elif white_piece == "queen":
              return get_queen_captures(white_position,board)
          else:
              return get_king_captures(white_position,board)

    except KeyError:
      return []


# Test cases for get_capturable_pieces
# Setup a comprehensive test board with various piece positions
test_board = {
    # White pieces
    "e4": {"piece": "pawn",   "colour": "white"},   # White pawn in middle
    "a1": {"piece": "rook",   "colour": "white"},   # White rook in corner
    "c1": {"piece": "bishop", "colour": "white"},   # White bishop
    "g1": {"piece": "knight", "colour": "white"},   # White knight
    "d1": {"piece": "queen",  "colour": "white"},   # White queen
    "e1": {"piece": "king",   "colour": "white"},   # White king

    # Black pieces (potential captures)
    "d5": {"piece": "bishop", "colour": "black"},   # Capturable by pawn diagonally
    "f5": {"piece": "knight", "colour": "black"},   # Capturable by pawn diagonally
    "e5": {"piece": "rook",   "colour": "black"},   # Not capturable by pawn (directly in front)

    "a3": {"piece": "knight", "colour": "black"},   # Capturable by bishop diagonally
    "a8": {"piece": "queen",  "colour": "black"},   # Capturable by rook vertically
    "h1": {"piece": "bishop", "colour": "black"},   # Capturable by rook horizontally

    "h6": {"piece": "rook",   "colour": "black"},   # Capturable by bishop diagonally

    "e3": {"piece": "pawn",   "colour": "black"},   # Capturable by knight L-move
    "f3": {"piece": "bishop", "colour": "black"},   # Capturable by knight L-move

    "b1": {"piece": "pawn",   "colour": "black"},   # Capturable by queen horizontally
    "d3": {"piece": "rook",   "colour": "black"},   # Capturable by queen diagonally
    "h4": {"piece": "knight", "colour": "black"},   # Capturable by queen diagonally

    "e2": {"piece": "bishop", "colour": "black"},   # Capturable by king adjacent
    "f2": {"piece": "pawn",   "colour": "black"},   # Capturable by king adjacent
}

# Test case 1: White pawn captures
pawn_captures = get_capturable_pieces(test_board, "pawn", "e4")
assert sorted(pawn_captures) == sorted(["d5", "f5"])
assert "e5" not in pawn_captures  # Pawn can't capture directly in front

# Test case 2: White rook captures
rook_captures = get_capturable_pieces(test_board, "rook", "a1")
assert sorted(rook_captures) == sorted(["a3",  "b1"])

# Test case 3: White bishop captures
bishop_captures = get_capturable_pieces(test_board, "bishop", "c1")
assert sorted(bishop_captures) == sorted(["a3", "e3"])

# Test case 4: White knight captures
knight_captures = get_capturable_pieces(test_board, "knight", "g1")
assert sorted(knight_captures) == sorted(["e2", "f3"])

# Test case 5: White queen captures
queen_captures = get_capturable_pieces(test_board, "queen", "d1")
assert sorted(queen_captures) == sorted(["d3", "e1", "c1", "e2"])

# Test case 6: White king captures
king_captures = get_capturable_pieces(test_board, "king", "e1")
assert sorted(king_captures) == sorted(["d1", "e2", "f2"])

# Test case 7: Invalid piece
assert get_capturable_pieces(test_board, "dragon", "e4") == []

# Test case 8: Invalid position
assert get_capturable_pieces(test_board, "pawn", "z9") == []

# Test case 9: Piece not at specified position
assert get_capturable_pieces(test_board, "bishop", "e4") == []

# Test case 10: Empty board
empty_board = {}
assert get_capturable_pieces(empty_board, "pawn", "e4") == []

print("All get_capturable_pieces test cases passed!")




All get_capturable_pieces test cases passed!


In [15]:
# Helper function to help with printing pretty chess pieces.
def get_chess_piece_symbol(piece: str, color: str) -> str:
    """
    Returns the UTF-8 symbol for the specified chess piece based on its name and color.

    Args:
        piece (str): The name of the chess piece (e.g., 'king', 'queen', 'rook', 'bishop', 'knight', 'pawn').
        color (str): The color of the piece ('white' or 'black').

    Returns:
        str: The UTF-8 symbol for the specified chess piece.

    Raises:
        ValueError: If the piece or color is invalid.
    """
    # Define a dictionary mapping piece names to their white and black UTF-8 symbols
    symbols = {
        "king": {"white": "\u2654", "black": "\u265A"},
        "queen": {"white": "\u2655", "black": "\u265B"},
        "rook": {"white": "\u2656", "black": "\u265C"},
        "bishop": {"white": "\u2657", "black": "\u265D"},
        "knight": {"white": "\u2658", "black": "\u265E"},
        "pawn": {"white": "\u2659", "black": "\u265F"},
    }

    symbols = {
        "king": {"white": " ♔ ", "black": " ♚ "},
        "queen": {"white": " ♕ ", "black": " ♛ "},
        "rook": {"white": " ♖ ", "black": " ♜ "},
        "bishop": {"white": " ♗ ", "black": " ♝ "},
        "knight": {"white": " ♘ ", "black": " ♞ "},
        "pawn": {"white": " ♙ ", "black": " ♟ "},
    }

    # Validate the piece and color inputs
    if piece not in symbols:
        raise ValueError(f"Invalid piece name: {piece}. Valid options are: {', '.join(symbols.keys())}.")
    if color not in symbols[piece]:
        raise ValueError(f"Invalid color: {color}. Valid options are: 'white' or 'black'.")

    # Return the corresponding symbol
    return symbols[piece][color]


In [16]:
def create_chessboard() -> dict:
    """
    Returns a dictionary with each key representing a place on the chessboard, all values have the value None

    Args:
        This function requires no arguments

    Returns:
        str: Dictionary of keys from a1 -> h8 containing None values (Piece, Colour)

    """
    columns = ["a","b","c","d","e","f","g","h"]
    board = {}

    for c in columns:
        for r in range(1, 9):
            key = f"{c}{r}"
            board[key] = {"piece": None, "colour": None, "column": c}

    ranks = list(range(8, 0, -1))
    rows = []

    for r in ranks:
        row = [r]
        for f in columns:
            cell = board[f"{f}{r}"]
            row.append(cell["piece"])
        rows.append(row)

    return board


In [17]:
 # OPTIONAL TASK 1: Implement a Nice Visualization Function
def print_chessboard(board: dict) -> list:
    """
    Returns a print tabulated list where the user can visualise the board dictionary as a 2D chessboard/

    Args:
        piece (str): The name of the chess piece (e.g., 'king', 'queen', 'rook', 'bishop', 'knight', 'pawn').
        board (dict): Dictionary which contains the chessboard data we wish to be printed. This can be for empty boards and populated dicts.

    Returns:
        list: printed list in tabulated format

    """
    columns = ["a","b","c","d","e","f","g","h"]

    for c in columns:
        for i in range(1, 9):
            key = f"{c}{i}"
            if key not in board:
                board[key] = {"piece": None, "colour": None}

    for position, values in board.items():
        values["column"] = position[0]
        piece  = values["piece"]
        colour = values["colour"]

        if piece and colour:
            values["symbol"] = get_chess_piece_symbol(piece, colour)
        else:
            values["symbol"] = None

    ranks = list(range(8, 0, -1))
    rows = []
    for r in ranks:
        row = [r]

        for f in columns:
            row.append(board[f"{f}{r}"]["symbol"])
        rows.append(row)

    return print(tabulate(rows, headers=columns, tablefmt="fancy_grid"))


# Test cases for get_capturable_pieces
# Setup a comprehensive test board with various piece positions
test_board = {
    # White pieces
    "e4": {"piece": "pawn",   "colour": "white"},   # White pawn in middle
    "a1": {"piece": "rook",   "colour": "white"},   # White rook in corner
    "c1": {"piece": "bishop", "colour": "white"},   # White bishop
    "g1": {"piece": "knight", "colour": "white"},   # White knight
    "d1": {"piece": "queen",  "colour": "white"},   # White queen
    "e1": {"piece": "king",   "colour": "white"},   # White king

    # Black pieces (potential captures)
    "d5": {"piece": "bishop", "colour": "black"},   # Capturable by pawn diagonally
    "f5": {"piece": "knight", "colour": "black"},   # Capturable by pawn diagonally
    "e5": {"piece": "rook",   "colour": "black"},   # Not capturable by pawn (directly in front)

    "a3": {"piece": "knight", "colour": "black"},   # Capturable by bishop diagonally
    "a8": {"piece": "queen",  "colour": "black"},   # Capturable by rook vertically
    "h1": {"piece": "bishop", "colour": "black"},   # Capturable by rook horizontally

    "h6": {"piece": "rook",   "colour": "black"},   # Capturable by bishop diagonally

    "e3": {"piece": "pawn",   "colour": "black"},   # Capturable by knight L-move
    "f3": {"piece": "bishop", "colour": "black"},   # Capturable by knight L-move

    "b1": {"piece": "pawn",   "colour": "black"},   # Capturable by queen horizontally
    "d3": {"piece": "rook",   "colour": "black"},   # Capturable by queen diagonally
    "h4": {"piece": "knight", "colour": "black"},   # Capturable by queen diagonally

    "e2": {"piece": "bishop", "colour": "black"},   # Capturable by king adjacent
    "f2": {"piece": "pawn",   "colour": "black"},   # Capturable by king adjacent
}


print(print_chessboard(test_board))



╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ a   │ b   │ c   │ d   │ e   │ f   │ g   │ h   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│  8 │ ♛   │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  7 │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  6 │     │     │     │     │     │     │     │ ♜   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  5 │     │     │     │ ♝   │ ♜   │ ♞   │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  4 │     │     │     │     │ ♙   │     │     │ ♞   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  3 │ ♞   │     │     │ ♜   │ ♟   │ ♝   │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  2 │     │     │     │     │ ♝   │ ♟   │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  1 │ ♖   │ ♟   │ ♗   │ ♕   │ ♔   │     │ ♘   │ ♝   │
╘════╧════

In [18]:
# OPTIONAL TASK : Implement Starting Position Generation
def random_start() -> str:
    """
    Created a randomised piece on the chessboard between a1 -> h8

    Args:
        This functions requires no arguments

    Returns:
        str: string in the required format needed to input onto the board. Ex: knight a4

    """
    letters = list('abcdefgh')
    position = str(choice(letters)) + str(randint(1,8))

    pieces = ["pawn","knight","bishop","rook","queen","king"]
    rand_piece = choice(pieces)

    return str(rand_piece) + " " + str(position)


In [19]:
# Mini-task 7: Main function where you reuse all previous functions and assemble working solution
def main() -> None:
    """
    Main function to handle user input, manage the board, and output capturable pieces.
    """

    print("\nWelcome, Would you like to enter the figures manually (m) or generate them randomly (r)?\n")
    start_choice = input("Enter r or m? ")

    if start_choice.strip().lower() == "m":
        print("Reminder these are your pieces:\n 'pawn', 'rook', 'knight', 'bishop', 'queen', 'king' ")

        board = create_chessboard()
        print_chessboard(board)

        # White piece input
        player_white_input = input("Enter white piece in 'piece place' (Ex: knight a4) format: ")

        while parse_piece_input(player_white_input) is None:
            player_white_input = input("Invalid input, try again: ")

        white_piece, white_position =  parse_piece_input(player_white_input)

        if add_piece(board, white_piece, white_position, "white") is True:
            print("Piece successfully added :) ")

        print_chessboard(board)


        # Black piece input
        player_black_input = input("Enter black piece in 'piece place' (Ex: knight a4) format: ")

        while parse_piece_input(player_black_input) is None:
            player_black_input = input("Invalid input, try again: ")

        black_piece, black_position =  parse_piece_input(player_black_input)

        if add_piece(board, black_piece, black_position, "black") is True:
            print("Piece successfully added :) ")
            print_chessboard(board)

        black_piece_count = 1
        while black_piece_count <= 16 or player_black_input != "done":
            if black_piece_count == 16:
                break

            player_black_input = input("Add another piece or finish by entering 'done': ")

            if player_black_input.strip() == "done":
                break

            while parse_piece_input(player_black_input) is None:
                player_black_input = input("Invalid input, try again: ")

            black_piece, black_position =  parse_piece_input(player_black_input)

            if add_piece(board, black_piece, black_position, "black") is False:
                pass
            else:
                print("Piece successfully added :) ")
                add_piece(board, black_piece, black_position, "black")
                black_piece_count += 1
                print_chessboard(board)

        board = {
            key: value
            for key, value in board.items()
            if value["piece"] is not None
        }

        if get_capturable_pieces(board, white_piece, white_position) == []:
            print("No black pieces are capturable ")
        else:
            print("With your",white_piece,"on tile", white_position," you can capture the pieces in the following tiles:")
            print(get_capturable_pieces(board, white_piece, white_position))

    # This section is the random path
    else:
        board = create_chessboard()

        # White input
        white_piece, white_position =  parse_piece_input(random_start())

        if add_piece(board, white_piece, white_position, "white") is False:
            white_piece, white_position =  parse_piece_input(random_start())

        # Black input
        black_piece, black_position =  parse_piece_input(random_start())

        if white_position == black_position:
            while white_position == black_position:
                black_piece, black_position =  parse_piece_input(random_start())

        counter = 1
        while counter <= 16:
            while add_piece(board, black_piece, black_position, "black") is False:
                black_piece, black_position =  parse_piece_input(random_start())
            counter += 1

        new_board = {
            k: v
            for k, v in board.items()
            if v["piece"] is not None
        }

        print_chessboard(board)
        print("With your",white_piece,"on tile", white_position," you can capture the pieces in the following tiles:")
        print(get_capturable_pieces(new_board, white_piece, white_position))

# Call main a function to run your chess program!

In [20]:
main()


Welcome, Would you like to enter the figures manually (m) or generate them randomly (r)?

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ a   │ b   │ c   │ d   │ e   │ f   │ g   │ h   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│  8 │ ♞   │ ♜   │     │     │     │     │     │ ♟   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  7 │     │     │ ♝   │ ♛   │     │ ♟   │ ♞   │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  6 │     │ ♜   │     │     │     │     │ ♜   │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  5 │     │     │     │     │     │     │ ♝   │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  4 │     │     │     │     │     │ ♞   │     │ ♜   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  3 │     │     │     │ ♛   │     │     │     │ ♛   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  2 │     │     │     │     │     │     │     │ ♟   │
├────┼─────┼─────┼─────┼─────

# Optional tasks
---
You have a choice to implement some optional tasks.
We have prepared a helper funtion to return the chess pieces as UTF-8 sybol, If utilized this would make your printing so much better
<br>
##♔  ♚
##♕  ♛
##♖  ♜
##♗  ♝
##♘  ♞
##♙  ♟



In [22]:
# Helper function to help with printing pretty chess pieces.
def get_chess_piece_symbol(piece: str, color: str) -> str:
    """
    Returns the UTF-8 symbol for the specified chess piece based on its name and color.

    Args:
        piece (str): The name of the chess piece (e.g., 'king', 'queen', 'rook', 'bishop', 'knight', 'pawn').
        color (str): The color of the piece ('white' or 'black').

    Returns:
        str: The UTF-8 symbol for the specified chess piece.

    Raises:
        ValueError: If the piece or color is invalid.
    """
    # Define a dictionary mapping piece names to their white and black UTF-8 symbols
    symbols = {
        "king": {"white": "\u2654", "black": "\u265A"},
        "queen": {"white": "\u2655", "black": "\u265B"},
        "rook": {"white": "\u2656", "black": "\u265C"},
        "bishop": {"white": "\u2657", "black": "\u265D"},
        "knight": {"white": "\u2658", "black": "\u265E"},
        "pawn": {"white": "\u2659", "black": "\u265F"},
    }

    symbols = {
        "king": {"white": " ♔ ", "black": " ♚ "},
        "queen": {"white": " ♕ ", "black": " ♛ "},
        "rook": {"white": " ♖ ", "black": " ♜ "},
        "bishop": {"white": " ♗ ", "black": " ♝ "},
        "knight": {"white": " ♘ ", "black": " ♞ "},
        "pawn": {"white": " ♙ ", "black": " ♟ "},
    }

    # Validate the piece and color inputs
    if piece not in symbols:
        raise ValueError(f"Invalid piece name: {piece}. Valid options are: {', '.join(symbols.keys())}.")
    if color not in symbols[piece]:
        raise ValueError(f"Invalid color: {color}. Valid options are: 'white' or 'black'.")

    # Return the corresponding symbol
    return symbols[piece][color]
